In [5]:
import requests
import json
import time
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta


In [ ]:
# Helium API had variety of functions to dive into network data, but for rewards we need only get_network_rewards
# meanwhile access is much easier via https://etl.dewi.org/, e.g. use this query to get rewards and num_accounts receiving those on daily basis:
# Select 
# cast(time/24/60/60 as INT)*24*60*60 as ts_day, 
# -- type, --optional to filter by type (add in group by hthen)
# count(distinct account) as num_accounts,
# sum(amount)/100000000 as amount_hnt 
# from rewards
# --where time>=1631000000 --filter by time if query times out
# --and time < 1633000000
# group by 1
#  order by 1 

# below has code on API calls
# API would sometimes fail when we try to get data for a full month, hence used weekly data then and then combined - output stored in df_net_rewards

In [2]:
def get_network_rewards(min_date,**kwargs):
    cursor = kwargs.get('cursor')
    max_date = kwargs.get('max_date',datetime.strftime(datetime.now(),'%Y-%m-%s'))
    bucket = kwargs.get('bucket','week')
    if bucket=='no':
        request_str='https://api.helium.io/v1/rewards/sum?min_time={min_time}&max_time={max_time}'.format(
        min_time=min_date,
        max_time=max_date
        )
    else:
        request_str='https://api.helium.io/v1/rewards/sum?min_time={min_time}&max_time={max_time}&bucket={bucket}'.format(
        min_time=min_date,
        max_time=max_date,    
        bucket=bucket
        )
    if cursor:
        request_str+='&cursor={cursor}'.format(cursor=cursor)
    print(request_str)
    return requests.get(request_str).json()

In [3]:
#not used for rewards output
def get_hotspots(**kwargs):
    cursor = kwargs.get('cursor')
    request_str='https://api.helium.io/v1/hotspots'
    if cursor:
        request_str+='?cursor={cursor}'.format(cursor=cursor)
    print(request_str)
    return requests.get(request_str).json()

def get_datacredit_burn(min_date,**kwargs):
    cursor = kwargs.get('cursor')
    max_date = kwargs.get('max_date',datetime.strftime(datetime.now(),'%Y-%m-%s'))
    bucket = kwargs.get('bucket','week')
    request_str='https://api.helium.io/v1/dc_burns/sum?min_time={min_time}&max_time={max_time}&bucket={bucket}'.format(
        min_time=min_date,
        max_time=max_date,    
        bucket=bucket
    )
    if cursor:
        request_str+='&cursor={cursor}'.format(cursor=cursor)
    print(request_str)
    return requests.get(request_str).json()

def get_hotspot_activity_count(address,**kwargs):
    cursor = kwargs.get('cursor')
    request_str='https://api.helium.io/v1/hotspots/{address}/roles/count'.format(
        address=address
    )
    if cursor:
        request_str+='?cursor={cursor}'.format(cursor=cursor)
    print(request_str)
    response = requests.get(request_str)
    # response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code == 200:
        return response.json()
    else:
        return {'error':response.status_code}
    

def get_hotspot_rewards(address,min_time,**kwargs):
    cursor = kwargs.get('cursor')
    max_time = kwargs.get('max_time',datetime.strftime(datetime.now(),'%Y-%m-%d'))
    request_str='https://api.helium.io/v1/hotspots/{address}/rewards?min_time={min_time}&max_time={max_time}'.format(
        min_time=min_time,
        max_time=max_time,
        address=address
    )
    if cursor:
        request_str+='&cursor={cursor}'.format(cursor=cursor)
    print(request_str)
    response = requests.get(request_str)
    # response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code == 200:
        return response.json()
    else:
        return {'error':response.status_code}

In [6]:
lst_month_start=[
'2019-08-01',
'2019-09-01',
'2019-10-01',
'2019-11-01',
'2019-12-01',
'2020-01-01',
'2020-02-01',
'2020-03-01',
'2020-04-01',
'2020-05-01',
'2020-06-01',
'2020-07-01',
'2020-08-01',
'2020-09-01',
'2020-10-01',
'2020-11-01',
'2020-12-01',
'2021-01-01',
'2021-02-01',
'2021-03-01',
'2021-04-01',
'2021-05-01',
'2021-06-01',
'2021-07-01',
'2021-08-01',
'2021-09-01',
'2021-10-01',
'2021-11-01',
'2021-12-01',
'2022-01-01',
'2022-02-01',
'2022-03-01',
'2022-04-01',
'2022-05-01',
'2022-06-01',
'2022-07-01',
'2022-08-01',
'2022-09-01',
'2022-10-01',
'2022-11-01',
'2022-12-01',
'2023-01-01',
'2023-02-01',
'2023-03-01',
'2023-04-01'
]
lst_month_end = [
'2019-08-31',
'2019-09-30',
'2019-10-31',
'2019-11-30',
'2019-12-31',
'2020-01-31',
'2020-02-28',
'2020-03-31',
'2020-04-30',
'2020-05-31',
'2020-06-30',
    '2020-07-31',
'2020-08-31',
'2020-09-30',
'2020-10-31',
'2020-11-30',
'2020-12-31',
'2021-01-31',
'2021-02-28',
'2021-03-31',
'2021-04-30',
'2021-05-31',
'2021-06-30',
'2021-07-31',
'2021-08-31',
'2021-09-30',
'2021-10-31',
'2021-11-30',
'2021-12-31',
'2022-01-31',
'2022-02-28',
'2022-03-31',
'2022-04-30',
'2022-05-31',
'2022-06-30',
'2022-07-31',
'2022-08-31',
'2022-09-30',
'2022-10-31',
'2022-11-30',
'2022-12-31',
'2023-01-31',
'2023-02-28',
'2023-03-31',
'2023-04-30'
]

In [ ]:
#below is for monthly data, to switch to weekly data, comment our ",bucket='no'" in line 20 and use same construction with week_interval, min_time_used and max_time_used as in code block for datacredit burn


dct_network_rewards = {
    'begin_date':[],
    'total':[],
    'median':[],
    'max':[],
    'avg':[]
}
no_result=0

for m in range(0, len(lst_month_end)):
    min_time_str = lst_month_start[m]
    max_time_str = lst_month_end[m]

    dct_burn=get_network_rewards(min_time_str,
                                  max_date=max_time_str
                                  ,bucket='no'
                                  )

    if 'error' in dct_burn.keys():
      print(dct_burn['error'])
      time.sleep(10)
      curs = ''  
      no_result+=1
    else:
      if 'total' in dct_burn['data'].keys():
            dct_network_rewards['total'].append(dct_burn['data']['total'])
            dct_network_rewards['begin_date'].append(min_time_str)
            if 'state_channel' in dct_burn['data'].keys():
              dct_network_rewards['median'].append(int(dct_burn['data']['median'])/1000000)
              dct_network_rewards['avg'].append(int(dct_burn['data']['avg'])/1000000)
              dct_network_rewards['max'].append(int(dct_burn['data']['max'])/1000000)
            else:
              dct_network_rewards['median'].append(0)
              dct_network_rewards['avg'].append(0)
              dct_network_rewards['max'].append(0)

      time.sleep(2)

In [ ]:
df_net_rewards=pd.DataFrame.from_dict(dct_network_rewards)
df_net_rewards.to_csv('net_rewards.csv')

In [153]:
#datacredit burn data, didn't use. amount_state_channel matches the revenues messari reports for Helium, see also https://docs.helium.com/blockchain/transaction-fees/ for details on the dc-burn categories
no_result=0
check = 0
curs = ''

dct_datacredit_burn = {
    'week':[],
    'amount_total':[],
    'amount_state_channel':[],
    'amount_fee':[],
    'amount_assert_location':[],
    'amount_add_gateway':[]
}

min_time = '2021-06-01' #no data before this date

week_interval=12
min_time_used = datetime.strptime(min_time,'%Y-%m-%d')
max_time_used = min_time_used + timedelta(days = 7*week_interval)

week_cnt = 0
while (min_time_used<datetime.now()):
    min_time_str = datetime.strftime(min_time_used,'%Y-%m-%d')
    if len(curs)>0:
        dct_burn=get_datacredit_burn(min_time_str,
                                  max_date=datetime.strftime(max_time_used,'%Y-%m-%d')
                                  ,cursor=curs)
    else:
        dct_burn=get_datacredit_burn(min_time_str,
                                  max_date=datetime.strftime(max_time_used,'%Y-%m-%d'))

    if 'error' in dct_burn.keys():
      print(dct_burn['error'])
      time.sleep(10)
      curs = ''  
      no_result+=1
    else:
      if len(dct_burn['data'])>0:
        for i in range(0,len(dct_burn['data'])):
            dct_datacredit_burn['week'].append(min_time_used+timedelta(days=7*i))
            if dct_burn['data'][i]['total']:
              dct_datacredit_burn['amount_total'].append(int(dct_burn['data'][i]['total'])/1000000)
            else:
              dct_datacredit_burn['amount_total'].append(0)
            if 'state_channel' in dct_burn['data'][i].keys():
              dct_datacredit_burn['amount_state_channel'].append(int(dct_burn['data'][i]['state_channel'])/1000000)
              dct_datacredit_burn['amount_fee'].append(int(dct_burn['data'][i]['fee'])/1000000)
              dct_datacredit_burn['amount_assert_location'].append(int(dct_burn['data'][i]['assert_location'])/1000000)
              dct_datacredit_burn['amount_add_gateway'].append(int(dct_burn['data'][i]['add_gateway'])/1000000)
            else:
              dct_datacredit_burn['amount_state_channel'].append(0)
              dct_datacredit_burn['amount_fee'].append(0)
              dct_datacredit_burn['amount_assert_location'].append(0)
              dct_datacredit_burn['amount_add_gateway'].append(0)
            week_cnt+=1
        check=1
        min_time_used = max_time_used
        max_time_used = min_time_used + timedelta(days = 7*week_interval)
      else:
        if 'cursor' in dct_burn.keys():
          curs = dct_burn['cursor']

    


https://api.helium.io/v1/dc_burns/sum?min_time=2018-01-01&max_time=2018-03-26&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2018-03-26&max_time=2018-06-18&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2018-06-18&max_time=2018-09-10&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2018-09-10&max_time=2018-12-03&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2018-12-03&max_time=2019-02-25&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2019-02-25&max_time=2019-05-20&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2019-05-20&max_time=2019-08-12&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2019-08-12&max_time=2019-11-04&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2019-11-04&max_time=2020-01-27&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2020-01-27&max_time=2020-04-20&bucket=week
https://api.helium.io/v1/dc_burns/sum?min_time=2020-04-20&max_time=2020-07-13&bucket=week
https://ap

In [154]:
df_dc_burn = pd.DataFrame.from_dict(dct_datacredit_burn)